In [18]:
import netket as nk
from numpy.lib.function_base import append
from itertools import permutations, combinations
from netket.vqs.mc import get_local_kernel_arguments, get_local_kernel
import time
import matplotlib.pyplot as plt
import json
import numpy as np
from numpy.linalg import eig
from flax.core.frozen_dict import FrozenDict
import flax
from netket.optimizer.qgt import QGTJacobianPyTree


from typing import Callable, Tuple
from functools import partial

import jax
from jax import numpy as jnp

from netket.stats import statistics as mpi_statistics, mean as mpi_mean, Stats
from netket.utils.types import PyTree
from netket.operator.spin import sigmax, sigmay,sigmaz

import matplotlib.pyplot as plt
from tqdm import tqdm
################################################################
from scipy.sparse.linalg import eigsh
################################################################
import os
import pandas as pd



In [27]:
# We always shorten netket as nk
import netket as nk

# Define a 1d chain
L = 6
g = nk.graph.Hypercube(length=L, n_dim=1, pbc=True)
hi = nk.hilbert.Spin(s=1/2, N=g.n_nodes)
# hamiltonian = nk.operator.Heisenberg(hilbert=hi, graph=g)
# hamiltonian = nk.operator.Ising(h=1.321, hilbert=hi, J=0.5, graph=g)
# hamiltonian = ha = sum([ sigmax(hi,i)*sigmax(hi,(i+1)%L)
#                        + sigmay(hi,i)*sigmay(hi,(i+1)%L)
#                        + sigmaz(hi,i)*sigmaz(hi,(i+1)%L)
#                          for i in range(L)])
# This creates an empty operator to which you can add others.
hamiltonian = nk.operator.LocalOperator(hi) 

# the list of nodes is given by g.nodes() 
for site in g.nodes():
    hamiltonian = hamiltonian - 1.0 * nk.operator.spin.sigmax(hi, site)

for (i,j) in g.edges():
    # you can multiply operators by using the @ operator
    hamiltonian = hamiltonian + nk.operator.spin.sigmaz(hi, i)@nk.operator.spin.sigmaz(hi, j)

In [28]:
# todelete
from scipy.sparse.linalg import eigsh
ha = hamiltonian.to_sparse()
e_gs, psi_gs = eigsh(ha, k=1)
e_gs = e_gs[0]
psi_gs = psi_gs.reshape(-1)
e_gs

7.727406610312544

The model itself is only a set of instructions on how to initialise the parameters and how to compute the result. 

To initialise the parameters we do the following:

In [29]:
def to_array(model, parameters):
    # begin by generating all configurations in the hilbert space.
    # all_States returns a batch of configurations that is (hi.n_states, N) large.
    all_configurations = hi.all_states()

    # now evaluate the model, and convert to a normalised wavefunction.
    logpsi = model.apply(parameters, all_configurations)
    psi = jnp.exp(logpsi)
    psi = psi / jnp.linalg.norm(psi)
    return psi

### 3.3 Energy
Now, using that function, let's build another one that computes the energy of the mean field state for the given parameters! 
To do that, you can for example convert the hamiltonian to a sparse format and compute the energy

In [49]:
def compute_energy(model, parameters, hamiltonian_sparse):
    psi_gs = to_array(model, parameters)
    return psi_gs.conj().T@(hamiltonian_sparse@psi_gs)

def compute_variance(model, parameters, hamiltonian_sparse):
    psi_gs = to_array(model, parameters)
    variance = psi_gs.conj().T@(hamiltonian_sparse@hamiltonian_sparse@psi_gs) - (psi_gs.conj().T@(hamiltonian_sparse@psi_gs))**2 
    return variance

In [31]:
# and like before, we can jit-compile it!
compute_energy_jit = jax.jit(compute_energy, static_argnames="model")

# and we precompute the sparse-hamiltonian to avoid the overhead of re-computing them all the time
hamiltonian_jax = hamiltonian.to_pauli_strings().to_jax_operator()
hamiltonian_sparse = hamiltonian.to_sparse()
hamiltonian_jax_sparse = hamiltonian_jax.to_sparse()

In [32]:
hamiltonian_jax

PauliStringsJax(hilbert=Spin(s=1/2, N=6), n_strings=12, dtype=float64, dict(operators:weights)=
    XIIIII : -1.0,
    IXIIII : -1.0,
    IIXIII : -1.0,
    IIIXII : -1.0,
    IIIIXI : -1.0,
    IIIIIX : -1.0,
    ZZIIII : 1.0,
    IZZIII : 1.0,
    IIIZZI : 1.0,
    IIZZII : 1.0,
    IIIIZZ : 1.0,
    ZIIIIZ : 1.0
)

In [33]:
from functools import partial

# we use partial to directly jit this function. Jitting the top-most will jit everything inside it as well.
@partial(jax.jit, static_argnames='model')
def compute_energy_and_gradient(model, parameters, hamiltonian_sparse):
    grad_fun = jax.value_and_grad(compute_energy, argnums=1)
    return grad_fun(model, parameters, hamiltonian_sparse)

In [51]:
model = nk.models.RBM(alpha=1)

parameters = model.init(jax.random.PRNGKey(0), np.ones((hi.size, )))

# logging: you can (if you want) use netket loggers to avoid writing a lot of boilerplate...
# they accumulate data you throw at them
logger = nk.logging.RuntimeLog()
from tqdm import tqdm
for i in tqdm(range(1000000)):
    # compute energy and gradient
    energy, gradient = compute_energy_and_gradient(model, parameters, hamiltonian_jax_sparse)
    
    # update parameters
    parameters = jax.tree_map(lambda x,y:x-0.01*y, parameters, gradient)
    
    # log energy: the logger takes a step argument and a dictionary of variables to be logged
    logger(step=i, item={'Energy':energy})

    
compute_variance(model, parameters, hamiltonian_sparse)    

100%|█████████████████████████████████| 100000/100000 [00:17<00:00, 5830.49it/s]


Array(0.00041161, dtype=float64)